### 2. Simulate a small company

a) Connect python to gemini, very important that you place the api key in .env and gitignore it

b) Use gemini to simulate 20 data points in json format containing the following fields: first_name, last_name, phone_number, email, department, salary, title.   
 See if you can prompt to direct the LLM output to have swedish names, phone numbers in swedish format (+46 731 29 52), departments (IT, HR, marketing, sales), reasonable salary (you might need to check some swedish statistics on salaries) and corresponding titles within these departments.

c) Now use pydantic to validate this json and put in proper schema that the fields should follow. You might need to do some processing such as removing backticks and maybe loading json data into a list with json.loads(). Also make sure that only correctly validated data should be stored.

d) Write this json data to a folder called output_data.

e) Use pandas to read the data as dataframe

f) Write a csv file to your output_data

g) Load this data into a staging layer and store this into a table called employees.

h) Use gemini to simulate departments data. There should be same departments as those you had in task b. Also add a description field and a contact person.

i) Add a departments table in your duckdb database under staging layer to store this data.


In [3]:
#a) Connect python to gemini, very important that you place the api key in .env and gitignore it

from google import genai

client = genai.Client()

b) 
Use gemini to simulate 20 data points in json format containing the following fields: 
- first_name, 
- last_name, 
- phone_number, 
- email, 
- department, 
- salary, 
- title
 See if you can prompt to direct the LLM output to have swedish names, phone numbers in swedish format (+46 731 29 52),   
 departments (IT, HR, marketing, sales), reasonable salary,   
 (you might need to check some swedish statistics on salaries) and corresponding titles within these departments.

In [4]:
def ask_LLM(prompt): 
    response = client.models.generate_content(
        model = "gemini-2.5-flash", 
        contents = prompt
    )
    return response.text

#ask_LLM("Skriv ett skämt")

In [5]:
response = ask_LLM("""
    Du ska ta fram en fil i json-format (ej markdown) som innehåller följande fält:
        first_name, last_name, phone_number, email, department, salary, title.
        Företaget har följande avdelningar(IT, HR, marketing, sales).
        Lönespannet är mellann 25000 kr per månad och 50000 kronor per månad. 
                   
    

    Exempel:
            {
                "first_name": "Johanna,
                "last_name": "Karlsson",
                "phone_number": "+46 700 123456",
                "email": johanna.karlsson@fortaget.se,
                "department": "HR", 
                "salary": 27800, 
                "title": "Löneadministratör
            }   
                   
    Ge mig en lista på 20 personer
""")

print(response)

[
    {
        "first_name": "Anna",
        "last_name": "Svensson",
        "phone_number": "+46 701 234567",
        "email": "anna.svensson@fortaget.se",
        "department": "HR",
        "salary": 32500,
        "title": "HR-Specialist"
    },
    {
        "first_name": "Johan",
        "last_name": "Karlsson",
        "phone_number": "+46 702 345678",
        "email": "johan.karlsson@fortaget.se",
        "department": "IT",
        "salary": 45000,
        "title": "Systemutvecklare"
    },
    {
        "first_name": "Maria",
        "last_name": "Larsson",
        "phone_number": "+46 703 456789",
        "email": "maria.larsson@fortaget.se",
        "department": "marketing",
        "salary": 38000,
        "title": "Marknadskoordinator"
    },
    {
        "first_name": "Erik",
        "last_name": "Johansson",
        "phone_number": "+46 704 567890",
        "email": "erik.johansson@fortaget.se",
        "department": "sales",
        "salary": 42000,
        "title"

c)   
Now use pydantic to validate this json and put in proper schema that the fields should follow.   
You might need to do some processing such as removing backticks and maybe loading json data into a list with json.loads().   
Also make sure that only correctly validated data should be stored.

In [6]:
from pydantic import BaseModel, EmailStr, field_validator
import json 

class Employee(BaseModel):
    first_name: str
    last_name: str
    phone_number: str  
    email: EmailStr    
    department: str
    salary: int
    title: str

    @field_validator('department')
    @classmethod
    def department_must_be_valid(cls, v):
        valid_departments = ['IT', 'HR', 'marketing', 'sales']
        if v not in valid_departments:
            raise ValueError(f'Invalid department: {v}')
        return v


class Department(BaseModel):
    department: str  
    department_address: str
    department_VAT: str
    description_field: str
    contact_person: str

    
class DepartmentList(BaseModel):
    objects: list[Department]

class EmployeeList(BaseModel):
    objects: list[Employee]
    



In [7]:
clean_response = response.replace('```json\n', '').replace('```', '')

print(clean_response)

[
    {
        "first_name": "Anna",
        "last_name": "Svensson",
        "phone_number": "+46 701 234567",
        "email": "anna.svensson@fortaget.se",
        "department": "HR",
        "salary": 32500,
        "title": "HR-Specialist"
    },
    {
        "first_name": "Johan",
        "last_name": "Karlsson",
        "phone_number": "+46 702 345678",
        "email": "johan.karlsson@fortaget.se",
        "department": "IT",
        "salary": 45000,
        "title": "Systemutvecklare"
    },
    {
        "first_name": "Maria",
        "last_name": "Larsson",
        "phone_number": "+46 703 456789",
        "email": "maria.larsson@fortaget.se",
        "department": "marketing",
        "salary": 38000,
        "title": "Marknadskoordinator"
    },
    {
        "first_name": "Erik",
        "last_name": "Johansson",
        "phone_number": "+46 704 567890",
        "email": "erik.johansson@fortaget.se",
        "department": "sales",
        "salary": 42000,
        "title"

In [8]:
try:
    json_data = json.loads(clean_response)

    employees = EmployeeList.model_validate({"objects": json_data})

    
    print("Data validerad utan fel!")

except json.JSONDecodeError as e:
    print(f"Ett fel uppstod vid laddning av JSON: {e}")
    # Skriv ut strängen för att se exakt vad som orsakade felet
    print(f"Ogiltig JSON-sträng: {clean_response}")
except ValueError as e:
    print(f"Valideringsfel: {e}")


Data validerad utan fel!


In [9]:
employees = EmployeeList.model_validate({"objects": json.loads(clean_response)})

employees

EmployeeList(objects=[Employee(first_name='Anna', last_name='Svensson', phone_number='+46 701 234567', email='anna.svensson@fortaget.se', department='HR', salary=32500, title='HR-Specialist'), Employee(first_name='Johan', last_name='Karlsson', phone_number='+46 702 345678', email='johan.karlsson@fortaget.se', department='IT', salary=45000, title='Systemutvecklare'), Employee(first_name='Maria', last_name='Larsson', phone_number='+46 703 456789', email='maria.larsson@fortaget.se', department='marketing', salary=38000, title='Marknadskoordinator'), Employee(first_name='Erik', last_name='Johansson', phone_number='+46 704 567890', email='erik.johansson@fortaget.se', department='sales', salary=42000, title='Säljare'), Employee(first_name='Sara', last_name='Andersson', phone_number='+46 705 678901', email='sara.andersson@fortaget.se', department='IT', salary=39000, title='IT-Tekniker'), Employee(first_name='Daniel', last_name='Nilsson', phone_number='+46 706 789012', email='daniel.nilsson@fo

d) 
Write this json data to a folder called output_data.

In [10]:
import os

folder_name = "output_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Mappen '{folder_name}' har skapats.")
else:
    print(f"Mappen '{folder_name}' existerar redan.")

Mappen 'output_data' existerar redan.


In [11]:
employees_dict = employees.model_dump()

In [12]:
file_path = os.path.join(folder_name, "employees.json")

In [13]:
with open(file_path, "w", encoding="utf-8") as file:  
    json.dump(employees_dict, file, indent=4)

print(f"Datat har sparats till {file_path}")

Datat har sparats till output_data\employees.json


e) 
Use pandas to read the data as dataframe

In [14]:
import pandas as pd

employees_list = employees_dict['objects']

df_employees = pd.DataFrame(employees_list)

# Skriv ut de första 5 raderna för att verifiera att det fungerar
df_employees.head()


,first_name,last_name,phone_number,email,department,salary,title
0,Anna,Svensson,+46 701 234567,anna.svensson@fortaget.se,HR,32500,HR-Specialist
1,Johan,Karlsson,+46 702 345678,johan.karlsson@fortaget.se,IT,45000,Systemutvecklare
2,Maria,Larsson,+46 703 456789,maria.larsson@fortaget.se,marketing,38000,Marknadskoordinator
3,Erik,Johansson,+46 704 567890,erik.johansson@fortaget.se,sales,42000,Säljare
4,Sara,Andersson,+46 705 678901,sara.andersson@fortaget.se,IT,39000,IT-Tekniker


f) 
Write a csv file to your output_data

In [15]:
folder_name = "output_data"
csv_file_path = os.path.join(folder_name, "employees.csv")

df_employees.to_csv(csv_file_path, index=False)

print(f"Datat har sparats till {csv_file_path}")

Datat har sparats till output_data\employees.csv


g) 
Load this data into a staging layer and store this into a table called employees.

In [ ]:
import dlt


@dlt.resource(write_disposition="replace", table_name="employees")
def load_data():
    data_list = df_employees.to_dict(orient="records")    
 
    for record in data_list:
        yield record

pipeline = dlt.pipeline(
    pipeline_name="employees", 
    destination="duckdb",
    dataset_name="staging"
)

load_info = pipeline.run(load_data())
print(load_info)

h) 
Use gemini to simulate departments data. There should be same departments as those you had in task b. Also add a description field and a contact person.


In [19]:
department = ask_LLM("""
    Du ska ta fram en fil i json-format (ej markdown) som innehåller följande fält:
       department, department_address, department_VAT, description_field, contact_person. 
        Företaget har följande avdelningar(IT, HR, marketing, sales).
        Jag vill inte behöva rensa json-filen så ge mig en json-fil utan backticks och annat.         
                   
    

    Exempel:
            {
                "department": "HR,
                "department_address": "Testvvägen 1, 413 18 Testeborg",
                "department_VAT": "123456-7890",
                "description_field": "HR hanterar företagets personalfrågor.",
                "contact_person": "Johanna Karlsson"
                
            }   
                   
    Ge mig en lista för företagets 4 avdelningar. 
""")

print(department)

[
  {
    "department": "IT",
    "department_address": "IT-avdelningen, Datahusvägen 1, 413 18 Testeborg",
    "department_VAT": "SE556000100001",
    "description_field": "IT-avdelningen ansvarar för företagets tekniska infrastruktur, systemutveckling och support.",
    "contact_person": "Erik Johansson"
  },
  {
    "department": "HR",
    "department_address": "HR-kontoret, Personalstigen 5, 413 18 Testeborg",
    "department_VAT": "SE556000100002",
    "description_field": "HR-avdelningen hanterar rekrytering, personalutveckling, löner och arbetsmiljöfrågor.",
    "contact_person": "Anna Svensson"
  },
  {
    "department": "Marketing",
    "department_address": "Marknadsavdelningen, Kreativallén 10, 413 18 Testeborg",
    "department_VAT": "SE556000100003",
    "description_field": "Marknadsavdelningen utvecklar och genomför marknadsföringsstrategier samt hanterar varumärkeskommunikation.",
    "contact_person": "Sara Lindgren"
  },
  {
    "department": "Sales",
    "department_

In [20]:
clean_department = department.replace('```json\n', '').replace('```', '')

clean_department


'[\n  {\n    "department": "IT",\n    "department_address": "IT-avdelningen, Datahusvägen 1, 413 18 Testeborg",\n    "department_VAT": "SE556000100001",\n    "description_field": "IT-avdelningen ansvarar för företagets tekniska infrastruktur, systemutveckling och support.",\n    "contact_person": "Erik Johansson"\n  },\n  {\n    "department": "HR",\n    "department_address": "HR-kontoret, Personalstigen 5, 413 18 Testeborg",\n    "department_VAT": "SE556000100002",\n    "description_field": "HR-avdelningen hanterar rekrytering, personalutveckling, löner och arbetsmiljöfrågor.",\n    "contact_person": "Anna Svensson"\n  },\n  {\n    "department": "Marketing",\n    "department_address": "Marknadsavdelningen, Kreativallén 10, 413 18 Testeborg",\n    "department_VAT": "SE556000100003",\n    "description_field": "Marknadsavdelningen utvecklar och genomför marknadsföringsstrategier samt hanterar varumärkeskommunikation.",\n    "contact_person": "Sara Lindgren"\n  },\n  {\n    "department": "

In [21]:
import json
try:
    json_data = json.loads(clean_department)

    department = DepartmentList.model_validate({"objects": json_data})

    
    print("Data validerad utan fel!")

except json.JSONDecodeError as e:
    print(f"Ett fel uppstod vid laddning av JSON: {e}")
    # Skriv ut strängen för att se exakt vad som orsakade felet
    print(f"Ogiltig JSON-sträng: {clean_response}")
except ValueError as e:
    print(f"Valideringsfel: {e}")

Data validerad utan fel!


In [22]:

department_dict = department.model_dump()

In [23]:
with open(file_path, "w", encoding="utf-8") as file:  
    json.dump(department_dict, file, indent=4)

print(f"Datat har sparats till {file_path}")

Datat har sparats till output_data\employees.json


In [24]:
import pandas as pd

department_list = department_dict['objects']

df_department = pd.DataFrame(department_list)

# Skriv ut de första 5 raderna för att verifiera att det fungerar
df_department.head()

,department,department_address,department_VAT,description_field,contact_person
0,IT,"IT-avdelningen, Datahusvägen 1, 413 18 Testeborg",SE556000100001,IT-avdelningen ansvarar för företagets teknisk...,Erik Johansson
1,HR,"HR-kontoret, Personalstigen 5, 413 18 Testeborg",SE556000100002,"HR-avdelningen hanterar rekrytering, personalu...",Anna Svensson
2,Marketing,"Marknadsavdelningen, Kreativallén 10, 413 18 T...",SE556000100003,Marknadsavdelningen utvecklar och genomför mar...,Sara Lindgren
3,Sales,"Säljkontoret, Kundvägen 7, 413 18 Testeborg",SE556000100004,"Säljavdelningen ansvarar för försäljning, kund...",Oskar Berg


i) 
Add a departments table in your duckdb database under staging layer to store this data.

In [25]:
import dlt


@dlt.resource(write_disposition="replace", table_name="departments")
def load_data():
    data_list = df_department.to_dict(orient="records")    
 
    for record in data_list:
        yield record

pipeline = dlt.pipeline(
    pipeline_name="departments", 
    destination="duckdb",
    dataset_name="staging"
)

load_info = pipeline.run(load_data())
print(load_info)

Pipeline departments load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset staging
The duckdb destination used duckdb:///c:\Users\susan\KodPython\ai_engineering_susanne_wenblad\exercises\Exercise3_pydantic\departments.duckdb location to store data
Load package 1757420409.4914744 is LOADED and contains no failed jobs
